## Bank Menü Aufgabe

In [13]:
# Startmenü Grundlagen

from __future__ import annotations                                                     # damit Typennotationen wie "List[Transaction]" funktionieren.
from dataclasses import dataclass, field                                               # für die transactions-Liste
from typing import List, Dict, Optional, Tuple                                         # für besser lesbaren Code
from datetime import datetime                                                          # hinterlässt Zeitstempel bei den Transaktionen
import itertools                                                                       # automatischer Zähler für Kontonummer und Kundennummer

@dataclass
class Transaction:
    timestamp: datetime
    type: str                                                                          # "Einzahlung", "Auszahlung, "Überweisung"
    amount: float
    description: str
    balance_after: float

@dataclass
class Customer:
    id: int
    vorname: str
    nachname: str
    adresse: str

@dataclass
class Account:
   account_number: int
   owner_id: int
   balance: float
   transactions: List[Transaction] = field(default_factory=list)

   def add_transaction(self, type_: str, amount: float, description: str = "") -> None:
      self.balance = round(self.balance, 2)
      t = Transaction(
         timestamp=datetime.now(),
         type=type_,
         amount=round(amount, 2),
         description=description,
         balance_after=round(self.balance, 2),
      )
      self.transactions.append(t)

def prompt_choice(prompt: str, options: Dict[str, str]) -> str:
    while True:
        print("\n" + prompt)
        for k, desc in options.items():
            print(f" [{k}] {desc}")
        choice = input("> ").strip().lower()
        if choice in options:
            return choice
        print("Ungültige Eingabe. Bitte erneut versuchen.")

def prompt_float(prompt: str, min_value: Optional[float] = None) -> float:
    while True:
        raw = input(prompt).strip().replace(",", ".")
        try:
            value = float(raw)
            if min_value is not None and value < min_value:
                print(f"Wert muss >= {min_value} sein.")
                continue
            return round(value, 2)
        except ValueError:
            print("Bitte gültige Zahl eingeben.")

def prompt_nonempty(prompt: str)-> str:
    while True:
        s = input(prompt).strip()
        if s:
            return s
        print("Eingabe darf nicht leer sein.")

In [14]:
# Banksystem für Kunden- & Kontoverwaltung

import itertools                                                                       # für automatische Zähler
from typing import Dict, Optional, List

class BankSystem:
    def __init__(self):
        self._customer_id_counter = itertools.count(1)                                 # Zähler für Kundennummer
        self._account_no_counter = itertools.count(100001)                             # Zähler für Kontonummer    
        self.customers: Dict[int, Customer] = {}
        self.accounts: Dict[int, Account] = {}
        self.customer_index: Dict[tuple[str, str, str,], int] = {}                     # Index für den Login (Vorname, Nachname, Adresse) -> customer_id

    # Kunden anlegen
    def create_customer(self, vorname: str, nachname: str, adresse: str) -> Customer:
        key = (vorname.strip().lower(), nachname.strip().lower(), adresse.strip().lower())
        if key in self.customer_index:
            cid = self.customer_index[key]
            return self.customers[cid]
        cid = next(self._customer_id_counter)
        customer = Customer(id=cid, vorname=vorname.strip(), nachname=nachname.strip(), adresse=adresse.strip())
        self.customers[cid] = customer
        self.customer_index[key] = cid
        return customer
    
    # Login-Vorgang
    def find_customer(self, vorname: str, nachname: str, adresse: str) -> Optional[Customer]:
        key = (vorname.strip().lower(), nachname.strip().lower(), adresse.strip().lower())
        cid = self.customer_index.get(key)
        return self.customers.get(cid) if cid else None
    
    # Anlegen eines Kundenkontos
    def create_account(self, customer_id: int) -> Account:
        acc_no = next(self._account_no_counter)
        acc = Account(account_number=acc_no, owner_id=customer_id, balance=0.0)
        self.accounts[acc_no] = acc
        return acc

    # gibt Liste aller Konten eines Kunden aus 
    def get_customer_accounts(self, customer_id: int) -> List[Account]:
        return [a for a in self.accounts.values() if a.owner_id == customer_id]
    
    # Findet Konto anhand von Kontonummer
    def get_account(self, account_number: int) -> Optional[Account]:
        return self.accounts.get(account_number)

In [15]:
# Kundenmenü

def customer_menu(customer: Customer, bank: BankSystem):
    while True:
        choice = prompt_choice(
            f" === KUNDEN-MENÜ (eingeloggt: {customer.vorname} {customer.nachname}) ===",
            {
                "1": "Details ausgeben",
                "2": "Bankkonto eröffnen",
                "3": "Zurück zum Start"
            }
        )

        if choice == "1":
            print(f"\nKundendetails:\n ID: {customer.id}\n Name: {customer.vorname} {customer.nachname}\n Adresse: {customer.adresse}")
            accounts = bank.get_customer_accounts(customer.id)
            if accounts:
                print("  Konten:", ", ".join(str(a.account_number) for a in accounts))
            else:
                print("  Konten: (noch keine)")

        elif choice == "2":
            accounts = bank.create_account(customer.id)
            print(f"\nNeues Konto erstellt. Kontonummer: {accounts.account_number}")

        elif choice == "3":
            print("\nZurück zum Startmenü.")
            break

In [16]:
# Kontomenü

def account_menu(account: Account, bank: BankSystem):
    while True:
        choice = prompt_choice(
            f"=== KONTO-MENÜ (Konto {account.account_number}) ===",
            {
                "1": "Einzahlen",
                "2": "Auszahlen",
                "3": "Überweisung",
                "4": "Kontoauszug",
                "5": "Kontodetails",
                "z": "Zurück zum Kundenmenü"
            }
        )

        if choice == "1":
            amount = prompt_float("Betrag einzahlen: ", min_value=0.01)
            account.balance += amount
            account.add_transaction("Einzahlung", amount, "Einzahlung")
            print(f"{amount} € eingezahlt. Neuer Kontostand: {account.balance} €")

        elif choice == "2":
            amount = prompt_float("Betrag auszahlen: ", min_value=0.01)
            if amount > account.balance:
                print("Fehler: Nicht genügend Guthaben.")
            else:
                account.balance -= amount
                account.add_transaction("Auszahlung", amount, "Auszahlung")
                print(f"{amount} € ausgezahlt. Neuer Kontostand: {account.balance} €")

        elif choice == "3":
            target_no = int(prompt_nonempty("Empfängerkontonummer: "))
            target_account = bank.get_account(target_no)
            if not target_account:
                print("Fehler: Empfängerkonto nicht gefunden.")
                continue
            amount = prompt_float("Betrag überweisen: ", min_value=0.01)
            if amount > account.balance:
                print("Fehler: Nicht genügend Guthaben.")
                continue
            account.balance -= amount
            account.add_transaction("Überweisung", amount, f"Überwiesen an {target_no}")
            target_account.balance += amount
            target_account.add_transaction("Überweisung", amount, f"Überwiesen von {account.account_number}")
            print(f"{amount} @ an Konto {target_no} überwiesen. Neuer Kontostand: {account.balance} €")

        elif choice == "4":
            if not account.transactions:
                print("Keine Transaktionen vorhanden.")
            else:
                print("\n--- Kontoauszug ---")
                for t in account.transactions:
                    print(f"{t.timestamp.strftime('%Y-%m-%d %H:%M:%S')} | {t.type} | {t.amount} € | {t.description} | Saldo: {t.balance_after} €")
                print("-------------------")

        elif choice == "5":
            print(f"\nKontonummer: {account.account_number}\nKontostand: {account.balance} €\nBesitzer-ID: {account.owner_id}")

        elif choice == "z":
            print("Zurück zum Kundenmenü.")
            break

In [17]:
# Hauptprogramm

def start_menu(bank: BankSystem):
    while True:
        choice = prompt_choice(
            "--- STARTMENÜ ---",
            {
                "1": "Kunde werden",
                "2": "Einloggen",
                "3": "System beenden"
            }
        )

        if choice == "1":
            vorname = prompt_nonempty("Vorname: ")
            nachname = prompt_nonempty("Nachname: ")
            adresse = prompt_nonempty("Adresse: ")
            kunde = bank.create_customer(vorname, nachname, adresse)
            print(f"\nKunde {kunde.vorname} {kunde.nachname} angelegt und eingeloggt.")
            customer_menu(kunde, bank)
        
        elif choice == "2":
            vorname = prompt_nonempty("Vorname: ")
            nachname = prompt_nonempty("Nachname: ")
            adresse = prompt_nonempty("Adresse: ")
            kunde = bank.find_customer(vorname, nachname, adresse)
            if kunde:
                print(f"\nWillkommen zurück, {kunde.vorname} {kunde.nachname}!")
                customer_menu(kunde, bank)
            else:
                print("Kunde nicht gefunden. Bitte erneut versuchen.")
        
        elif choice == "3":
            print("System wird beendet.")
            break

## Funktions-Tests

In [18]:
# Bank System initialisieren

bank = BankSystem()
print("Bank-System initialisiert.")

Bank-System initialisiert.


In [19]:
# Kunden anlegen

kunde1 = bank.create_customer("Clara", "Test", "Musterstraße 3")
kunde2 = bank.create_customer("Max", "Muster", "Beispielweg 5")

print("Kunden angelegt:")
print(kunde1)
print(kunde2)

# Kunden-Index ausgeben
print("\nAktueller Kunden-Index:")
print(bank.customer_index)

Kunden angelegt:
Customer(id=1, vorname='Clara', nachname='Test', adresse='Musterstraße 3')
Customer(id=2, vorname='Max', nachname='Muster', adresse='Beispielweg 5')

Aktueller Kunden-Index:
{('clara', 'test', 'musterstraße 3'): 1, ('max', 'muster', 'beispielweg 5'): 2}


In [20]:
# Kunden Login testen

login1 = bank.find_customer("Clara", "Test", "Musterstraße 3")
login2 = bank.find_customer("Max", "Muster", "Beispielweg 5")
login3 = bank.find_customer("Nicht", "Existiert", "Keine Adresse")

print("Login-Ergebnisse:")
print("Clara:", login1)
print("Max:", login2)
print("Nicht vorhanden:", login3)

Login-Ergebnisse:
Clara: Customer(id=1, vorname='Clara', nachname='Test', adresse='Musterstraße 3')
Max: Customer(id=2, vorname='Max', nachname='Muster', adresse='Beispielweg 5')
Nicht vorhanden: None


In [21]:
# Konten erstellen

konto1 = bank.create_account(kunde1.id)
konto2 = bank.create_account(kunde2.id)

print(f"Konto von Clara: {konto1.account_number}")
print(f"Konto von Max: {konto2.account_number}")

Konto von Clara: 100001
Konto von Max: 100002


In [22]:
# Einzahlen und Auszahlen testen

konto1.balance += 500.0
konto1.add_transaction("Einzahlung", 500.0, "Startguthaben")
konto2.balance += 300.0
konto2.add_transaction("Einzahlung", 300.0, "Startguthaben")

print(f"Claras Kontostand: {konto1.balance} €")
print(f"Max' Kontostand: {konto2.balance} €")



betrag = 100
if betrag <= konto1.balance:
    konto1.balance -= betrag
    konto1.add_transaction("Auszahlung", betrag, "Testauszahlung")

print(f"Claras Kontostand nach Auszahlung: {konto1.balance} €")

Claras Kontostand: 500.0 €
Max' Kontostand: 300.0 €
Claras Kontostand nach Auszahlung: 400.0 €


In [23]:
# Überweisung testen

betrag = 200
if betrag <= konto1.balance:
    konto1.balance -= betrag
    konto1.add_transaction("Überweisung", betrag, f"Überweisung an {konto2.account_number}")

    konto2.balance += betrag
    konto2.add_transaction("Überweisung", betrag, f"Überweisung von {konto1.account_number}")

print(f"Claras Kontostand: {konto1.balance} €")
print(f"Max Kontostand: {konto2.balance} €")

Claras Kontostand: 200.0 €
Max Kontostand: 500.0 €


In [24]:
# Kontoauszug anzeigen

def show_transactions(account: Account):
    print(f"\n--- Kontoauszug Konto {account.account_number} ---")
    for t in account.transactions:
        print(f"{t.timestamp.strftime('%Y-%m-%d %H:%M:%S')} | {t.type} | {t.amount} € | {t.description} | Saldo: {t.balance_after} €")
    print("-------------------")

show_transactions(konto1)
show_transactions(konto2)


--- Kontoauszug Konto 100001 ---
2025-09-02 14:22:59 | Einzahlung | 500.0 € | Startguthaben | Saldo: 500.0 €
2025-09-02 14:22:59 | Auszahlung | 100 € | Testauszahlung | Saldo: 400.0 €
2025-09-02 14:22:59 | Überweisung | 200 € | Überweisung an 100002 | Saldo: 200.0 €
-------------------

--- Kontoauszug Konto 100002 ---
2025-09-02 14:22:59 | Einzahlung | 300.0 € | Startguthaben | Saldo: 300.0 €
2025-09-02 14:22:59 | Überweisung | 200 € | Überweisung von 100001 | Saldo: 500.0 €
-------------------
